# Prediction of Disease in Casava Plants

## Retrain exisiting model to update weights

# Load needed librarys and the data sets.

In [ ]:
#Load Nessicery Librarys
import numpy as np 
import pandas as pd 

import cv2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
# Any results you write to the current direct



#ory are saved as output.

from time import time

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16

from tensorflow.keras.callbacks import TensorBoard

import keras
from keras.models import Sequential
from keras.models import model_from_json
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

import math

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/Prediction of Disease in Cassava Plants. /Cassava/train.csv",dtype=str)
train_path = "/content/drive/MyDrive/Prediction of Disease in Cassava Plants. /Cassava/train_images"
test = pd.read_csv("/content/drive/MyDrive/Prediction of Disease in Cassava Plants. /Cassava/sample_submission.csv",dtype=str)
test_path = "/content/drive/MyDrive/Prediction of Disease in Cassava Plants. /Cassava/test_images"
print(train.head())

         image_id label
0  1000015157.jpg     0
1  1000201771.jpg     3
2   100042118.jpg     1
3  1000723321.jpg     1
4  1000812911.jpg     3


# Data generators. 

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.25,
                                  rotation_range=10, # rotation
                                  width_shift_range=0.2, # horizontal shift
                                  height_shift_range=0.2, # vertical shift
                                  zoom_range=0.2) # zoom
Val_datagen = ImageDataGenerator(rescale=1/255, validation_split=0.25)
test_datagen = ImageDataGenerator(rescale=1/255)

In [ ]:
bs = 256

train_generator = train_datagen.flow_from_dataframe(
    dataframe = train,
    directory = train_path,
    x_col = "image_id",
    y_col = "label",
    subset = "training",
    batch_size = bs,
    #seed = 1,
    shuffle = True,
    class_mode = "categorical",
    target_size = (150,200))

valid_generator = Val_datagen.flow_from_dataframe(
    dataframe = train,
    directory = train_path,
    x_col = "image_id",
    y_col = "label",
    subset = "validation",
    batch_size = bs,
    #seed = 1,
    shuffle = True,
    class_mode = "categorical",
    target_size = (150,200))

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = test_path,
    x_col = "image_id",
    y_col = None,
    batch_size = bs,
    seed = 1,
    shuffle = False,
    class_mode = None,
    target_size = (150,200))

Found 16048 validated image filenames belonging to 5 classes.
Found 5349 validated image filenames belonging to 5 classes.
Found 1 validated image filenames.


In [ ]:
tr_size = 16048
va_size = 5349
te_size = 1
tr_steps = math.ceil(tr_size / bs)
va_steps = math.ceil(va_size / bs)
te_steps = math.ceil(te_size / bs)

# Load the existing model. 

In [ ]:
cnn_model = keras.models.load_model('/content/drive/MyDrive/Prediction of Disease in Cassava Plants. /my_model_v06.h5' )
cnn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 200, 64)      4864      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 150, 200, 64)      102464    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 100, 64)       0         
_________________________________________________________________
batch_normalization (BatchNo (None, 75, 100, 64)       256       
_________________________________________________________________
dropout (Dropout)            (None, 75, 100, 64)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 75, 100, 128)      73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 75, 100, 128)      1

WHat his is trying to do is to see if we can get a better model form the one that we currently have. So running this will take the weights from the previous model and will then retrain that model using the training and validation set to see that we can make better preditcion with the data. 

In [ ]:
%%time 

opt = keras.optimizers.Adam(0.01)
cnn_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

h1 = cnn_model.fit(train_generator, steps_per_epoch=tr_steps, epochs=20,
                       validation_data=valid_generator, validation_steps=va_steps)

# Generate Accuracy and loss charts to check how well the model is doing. 

In [ ]:
start = 1
ep_rng = np.arange(start,len(h1.history['accuracy']))

plt.figure(figsize=[12,6])
plt.subplot(1,2,1)
plt.plot(ep_rng, h1.history['accuracy'][start:], label='Training Accuracy')
plt.plot(ep_rng, h1.history['val_accuracy'][start:], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1,2,2)
plt.plot(ep_rng, h1.history['loss'][start:], label='Training Loss')
plt.plot(ep_rng, h1.history['val_loss'][start:], label='Validation Loss')
plt.xlabel('Epoch')
plt.legend()

plt.show()

# Save the weights for the new model. 

In [ ]:
#Save Model Weights and Configuration
cnn_model.save('models/my_model_retrain_v01.h5')